In [8]:
import pandas as pd
import numpy as np
import os
import tqdm
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
from os.path import join as pjoin

In [4]:
from caveclient import CAVEclient

client = CAVEclient()
v1dd_client = CAVEclient(server_address="https://global.em.brain.allentech.org")
v1dd_client.auth.save_token(token="b6459ccf83fb10a3cea2f122eb704e0f",
                            overwrite=True)

client = CAVEclient(datastack_name='v1dd_public',)
client.info.get_datastack_info()


{'aligned_volume': {'id': 3,
  'name': 'v1dd',
  'display_name': None,
  'description': 'the first fine alignment of the v1dd dataset',
  'image_source': 'precomputed://gs://v1dd_imagery/image/aligned_image'},
 'segmentation_source': 'graphene://https://api.em.brain.allentech.org/segmentation/table/v1dd_public',
 'skeleton_source': 'precomputed://middleauth+https://api.em.brain.allentech.org/skeletoncache/api/v1/v1dd_public/precomputed/skeleton',
 'analysis_database': None,
 'viewer_site': 'spelunker.cave-explorer.org',
 'synapse_table': 'synapses_v1dd',
 'soma_table': 'nucleus_detection_v0',
 'local_server': 'https://api.em.brain.allentech.org',
 'description': 'This is the public release of the v1dd dataset.',
 'viewer_resolution_x': 9.0,
 'viewer_resolution_y': 9.0,
 'viewer_resolution_z': 45.0,
 'proofreading_status_table': 'proofreading_status_and_strategy',
 'cell_identification_table': None,
 'proofreading_review_table': None}

In [10]:
data_dir = '/Users/rodrigo.gonzalezlaiz/Desktop/v1dd_1196'
mat_version = '1196'
# Loads cells with axon and dendrite proofreading
dendrite_proof_root_ids = np.load(
    pjoin(data_dir, f"proofread_dendrite_list_{mat_version}.npy"))
axon_proof_root_ids = np.load(
    pjoin(data_dir, f"proofread_axon_list_{mat_version}.npy"))

# get the ids of all proofread cells with both axon and dendrite proofreading
proof_root_ids = axon_proof_root_ids[np.isin(axon_proof_root_ids,
                                             dendrite_proof_root_ids)]

print(
    f"There are {len(dendrite_proof_root_ids)} cells with acceptable dendrites, and {len(axon_proof_root_ids)} cells with axon proofreading"
)
print(
    f"The number of cells with usable axons and dendrites is: {len(proof_root_ids)}"
)

There are 63986 cells with acceptable dendrites, and 1210 cells with axon proofreading
The number of cells with usable axons and dendrites is: 1148


In [14]:
coregistered_info = pd.read_feather(f"{data_dir}/coregistration_1196.feather")

In [15]:
coregistered_info

,pt_root_id,column,volume,plane,roi
0,864691132830842994,1,3,0,143
1,864691132741466457,1,3,2,40
2,864691132770893729,1,3,3,98
3,864691132770893729,1,3,3,98
4,864691132999240117,1,3,0,100
...,...,...,...,...,...
566,864691132744008540,1,3,2,236
567,864691132810202401,1,3,2,383
568,864691132925592361,1,3,2,393
569,864691132884727983,1,3,0,89


In [19]:
dendrite_and_coregistered_root_ids = dendrite_proof_root_ids[np.isin(
    dendrite_proof_root_ids, coregistered_info['pt_root_id'])]

In [21]:
syn_dfs = []
syn_path = f"{data_dir}/syn_df_dendrite_and_coregistered_{mat_version}.feather"
for post_root_ids in tqdm.tqdm(
        np.array_split(dendrite_and_coregistered_root_ids,
                       len(dendrite_and_coregistered_root_ids) // 40)):
    syn_df_chunk = client.materialize.query_table(
        "synapses_v1dd",
        filter_in_dict={"post_pt_root_id": post_root_ids},
        desired_resolution=[1, 1, 1],
        split_positions=True,
    )
    syn_df_chunk = syn_df_chunk[syn_df_chunk["pre_pt_root_id"] !=
                                syn_df_chunk["post_pt_root_id"]]
    syn_df_chunk = syn_df_chunk.drop(columns=[
        "created",
        "valid",
        "superceded_id",
        "pre_pt_supervoxel_id",
        "post_pt_supervoxel_id",
    ])
    assert len(syn_df_chunk) < 500_000
    syn_dfs.append(syn_df_chunk)

post_syn_df = pd.concat(syn_dfs)
post_syn_df.to_feather(syn_path, compression="zstd")

100%|██████████| 12/12 [18:12<00:00, 91.06s/it]


In [22]:
post_syn_df.shape

(2017256, 13)